In [2]:
import pandas as pd
from classes.mapping import neighborhood_mapping
import numpy as np

In [2]:
cases = pd.read_csv('data/encampment_cases_filtered.csv')
cases.dropna(subset=['Neighborhood'], inplace=True)
cases.drop(columns='Unnamed: 0', inplace=True)

In [3]:
cases['Mapped Neighborhood'] = cases['Neighborhood'].map(neighborhood_mapping)

In [4]:
neighborhood_counts = cases['Mapped Neighborhood'].value_counts()
neighborhood_counts = neighborhood_counts.reset_index()
neighborhood_counts.columns = ['Neighborhood', 'Homeless Cases']
neighborhood_counts

,Neighborhood,Homeless Cases
0,South of Market,4339
1,Mission,2625
2,Tenderloin,2047
3,Financial District/South Beach,1359
4,Western Addition,1238
5,Hayes Valley,1163
6,Golden Gate Park,756
7,Castro/Upper Market,620
8,Bayview Hunters Point,619
9,Potrero Hill,437


In [5]:
master_data = pd.read_csv('data/Neighborhood_master_table_by_geo_2020_2010.csv')
master_merged = pd.merge(master_data, neighborhood_counts, on = 'Neighborhood', how = 'left')

In [6]:
master_merged.fillna({'Homeless Cases' : 0}, inplace=True)
master_merged.loc[41, 'Homeless Cases'] = master_merged['Homeless Cases'].sum()

In [7]:
master_merged.to_csv('data/master_merged.csv', index=False)

In [9]:
## Load in master merged to make it counts & merge with region data
master_merged = pd.read_csv('data/master_merged.csv', index_col=False)
master_merged['Homeless concern rate'] = master_merged['Homeless Cases'] / master_merged['Total Population']
master_merged.loc[5, 'Neighborhood'] = 'Financial District/South Beach'
#master_merged.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'], inplace = True)

In [10]:
# Calcuated log of expected cases 
master_merged['Log Expected Cases'] = np.log(master_merged['Total Population'] * 18409 / 874784)

In [11]:
# Manually impute median household income for Lincoln Park from 2010 since 2020 is missing
master_merged.loc[14,'Median Household Income'] = 94792

In [12]:
master_merged.to_csv('data/master_merged.csv', index=False)